# Differential Equations Practicum
The purpose of this activity is to implement and/or understand the principles of numerical methods in differential equations.

Variant 24.

Original Equation:

$$
y' = xy^2-3xy,\:y(0)=2\\
\dfrac{y'}{y^2}=x-\dfrac{3x}{y}\\
z=1/y,\:dz=-\dfrac{dy}{y^2}\\
-z'=x(1-3z)\\
\dfrac{z'}{3z-1}=x\\
\dfrac{\ln{(3z-1)}}{3}=\dfrac{x^2}{2}+C_1\\
-1+3z=C_2e^{\dfrac{3x^2}{2}}\\
\dfrac{3}{y}=C_2e^{\frac{3x^2}{2}}+1\\
y=\dfrac{3}{1+C_2e^{\frac{3x^2}{2}}}\\
For\:y(0)=2\:=>\:2=\dfrac{3}{C_2+1}\:=>\:C_2=\frac{1}{2}
$$

Exact Solution:

$$
y=\frac{6}{2+e^{\frac{3x^2}{2}}}
$$



In [1]:
from math import exp
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

## Definition of original and exact solution functions

In [2]:
def my_func(x, y):
    return x * y * y - 3 * x * y

In [3]:
def exact_func(x,y):
    return 6/(exp(3*x*x/2)+2)

## Euler Method
bla bla blad

In [4]:
# Get function, initial value for x and y, end of the interval [x,X0] and number of steps to perform
def euler(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays
    x = [x0]
    y = [y0]
    # Calculate the step for x
    h = (X - x0) / N_steps
    
    # Perform the Euler method storing values that we get
    for i in range(N_steps):
        # x(i+1)=xi+h
        x.append(x[i] + h)
        
        # y(i+1)=xi+h*f(xi,yi)
        y.append(y[i] + h * func(x[i], y[i]))
    return x, y

## Improved Euler Method
Bla bla bla

In [5]:
# Get function, initial value for x and y, end of the interval [x,X0] and number of steps to perform
def improved_euler(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays
    x = [x0]
    y = [y0]
    
    # Calculate the step for x
    h = (X - x0) / N_steps
    
    # Perfrom improved Euler method storing values that we get
    for i in range(N_steps):
        x.append(x[i] + h)
        m1 = func(x[i], y[i])
        m2 = func(x[i + 1], y[i] + h * m1)
        y.append(y[i] + h * (m1 + m2) / 2)
    return x, y

## Runge Kutta Method
bla bla bla

In [6]:
# Get function, initial value for x and y, end of the interval [x,X0] and number of steps to perform
def runge_kutta(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays  
    x = [x0]
    y = [y0]
    
    # Calculate the step for x
    h = (X - x0) / N_steps
    
    # Perfrom Runge-kutta method storing values that we get
    for i in range(N_steps):
        x.append(x[i] + h)
        k1 = h * func(x[i], y[i])
        k2 = h * func(x[i] + h / 2, y[i] + k1 / 2)
        k3 = h * func(x[i] + h / 2, y[i] + k2 / 2)
        k4 = h * func(x[i] + h, y[i] + k3)
        y.append(y[i] + (k1 + 2 * k2 + 2 * k3 + k4) / 6)
    return x, y

In [7]:
x0 = 0
y0 = 2
X = 6.4
N_steps = 60

In [8]:
methods = {'euler':euler,'improved_euler':improved_euler, 'runge_kutta':runge_kutta}
pairs = []
for method in methods.values():
    pairs.append(method(my_func,x0,y0,X,N_steps))

In [9]:
p = figure(plot_width=400, plot_height=400)
output_notebook()

p.multi_line([pairs[0][0], pairs[1][0], pairs[2][0]],[pairs[0][1], pairs[1][1], pairs[2][1]],
             color=['#FF000','#00FF00','#0000FF'],alpha=[0.8, 0.8, 0.8], line_width=2)
show(p)

Loading BokehJS ...